In [1]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import re

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize, sent_tokenize

import keras
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, LSTM, SpatialDropout1D, Input, Bidirectional,Dropout
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# import data
df = pd.read_csv('dara.csv')
df = df.rename(columns={'0': 'title', '1': 'body', '2': 'tags'})
df['text'] = df['Title'].apply(lambda s: BeautifulSoup(s).text) + ' - ' + df['Body'].apply(lambda s: BeautifulSoup(s).text)

/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [4]:
df

,Title,Body,Tags,text
0,efficient way to implement paging,<p>Should I use LINQ's <code>Skip()</code> and...,<sql><sql-server><asp.net-mvc><linq-to-sql><pa...,efficient way to implement paging - Should I u...
1,What does variable names beginning with _ mean?,<p>When writing my first asp.net MVC applicati...,<c#><asp.net><syntax><variables><naming-conven...,What does variable names beginning with _ mean...
2,Is it valid to replace http:// with // in a <s...,<p>I have the following element:</p>\n\n<pre><...,<html><http><https><uri><protocol-relative>,Is it valid to replace http:// with // in a -...
3,Will new return NULL in any case?,<p>I know that according to C++ standard in ca...,<c++><visual-c++><memory-management><new-opera...,Will new return NULL in any case? - I know tha...
4,Convert float to string in positional format (...,<p>I want to print some floating point numbers...,<python><python-3.x><floating-point><number-fo...,Convert float to string in positional format (...
...,...,...,...,...
17667,Cross-Origin-Opener-Policy policy would block ...,<p>I am using firebase and its google auth too...,<firebase><next.js><oauth-2.0><firebase-authen...,Cross-Origin-Opener-Policy policy would block ...
17668,VSC PowerShell. After npm updating packages .p...,<p>I design websites in VSC and PowerShell is ...,<firebase><powershell><vue.js><npm><visual-stu...,VSC PowerShell. After npm updating packages .p...
17669,How to force GCC to assume that a floating-poi...,<p>There are cases where you know that a certa...,<c++><gcc><assembly><floating-point><micro-opt...,How to force GCC to assume that a floating-poi...
17670,Jupyter command `jupyter-lab` not found,<p>I have tried to install jupyter lab on my K...,<python><python-3.x><jupyter-notebook><jupyter...,Jupyter command `jupyter-lab` not found - I ha...


In [5]:
nltk.download('stopwords')
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer which is developed in recent years
stop=set(stopwords.words('english'))

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marzime/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def clean_txt(data, column: str):
    i=0
    str1=' '
    final_string=[]
    s=''
    for sent in data[column].values:
        filtered_sentence=[]
        #print(sent);
        sent=cleanhtml(sent) # remove HTMl tags
        for w in sent.split():
            for cleaned_words in cleanpunc(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue
        str1 = b" ".join(filtered_sentence)

        final_string.append(str1)
        i+=1

    data[f'clean_{column}']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review
    data[f'clean_{column}']=data[f'clean_{column}'].str.decode("utf-8")

    return data

In [7]:
df = clean_txt(df, 'text')

In [8]:
df['cleaned_tags'] = df['Tags'].replace('<','', regex=True)
df['cleaned_tags'] = df['cleaned_tags'].replace('>',' ', regex=True)

In [9]:
df

,Title,Body,Tags,text,clean_text,cleaned_tags
0,efficient way to implement paging,<p>Should I use LINQ's <code>Skip()</code> and...,<sql><sql-server><asp.net-mvc><linq-to-sql><pa...,efficient way to implement paging - Should I u...,effici way implement page use linq skip take m...,sql sql-server asp.net-mvc linq-to-sql paginat...
1,What does variable names beginning with _ mean?,<p>When writing my first asp.net MVC applicati...,<c#><asp.net><syntax><variables><naming-conven...,What does variable names beginning with _ mean...,variabl name begin mean write first asp net mv...,c# asp.net syntax variables naming-conventions
2,Is it valid to replace http:// with // in a <s...,<p>I have the following element:</p>\n\n<pre><...,<html><http><https><uri><protocol-relative>,Is it valid to replace http:// with // in a -...,valid replac follow case site https site may a...,html http https uri protocol-relative
3,Will new return NULL in any case?,<p>I know that according to C++ standard in ca...,<c++><visual-c++><memory-management><new-opera...,Will new return NULL in any case? - I know tha...,new return null case know accord standard case...,c++ visual-c++ memory-management new-operator ...
4,Convert float to string in positional format (...,<p>I want to print some floating point numbers...,<python><python-3.x><floating-point><number-fo...,Convert float to string in positional format (...,convert float string posit format without scie...,python python-3.x floating-point number-format...
...,...,...,...,...,...,...
17667,Cross-Origin-Opener-Policy policy would block ...,<p>I am using firebase and its google auth too...,<firebase><next.js><oauth-2.0><firebase-authen...,Cross-Origin-Opener-Policy policy would block ...,polici would block window close call error use...,firebase next.js oauth-2.0 firebase-authentica...
17668,VSC PowerShell. After npm updating packages .p...,<p>I design websites in VSC and PowerShell is ...,<firebase><powershell><vue.js><npm><visual-stu...,VSC PowerShell. After npm updating packages .p...,vsc powershel npm updat packag cannot load run...,firebase powershell vue.js npm visual-studio-c...
17669,How to force GCC to assume that a floating-poi...,<p>There are cases where you know that a certa...,<c++><gcc><assembly><floating-point><micro-opt...,How to force GCC to assume that a floating-poi...,forc gcc assum express case know certain expre...,c++ gcc assembly floating-point micro-optimiza...
17670,Jupyter command `jupyter-lab` not found,<p>I have tried to install jupyter lab on my K...,<python><python-3.x><jupyter-notebook><jupyter...,Jupyter command `jupyter-lab` not found - I ha...,jupyt command found tri instal jupyt lab kubun...,python python-3.x jupyter-notebook jupyter jup...


In [10]:
vectorizer_tf = TfidfVectorizer()
vectorizer_tf.fit(df['cleaned_tags'])
# print(sorted(vectorizer_tf.vocabulary_))
vector_tf = vectorizer_tf.transform([df['cleaned_tags'][0]])
print(vector_tf.shape)
print(vector_tf.toarray())

(1, 6943)
[[0. 0. 0. ... 0. 0. 0.]]


# Bag of words

In [18]:
vectorizer = CountVectorizer()
vectorizer.fit(df['clean_text'])
# print(sorted(vectorizer.vocabulary_))
vector = vectorizer.transform([df['clean_text'][0]])
print(vector.shape)
print(vector.toarray())

(1, 44518)
[[0 0 0 ... 0 0 0]]


In [20]:
vectorizer_ha = HashingVectorizer(n_features=6)
vectorizer_ha.fit(df['clean_text'])
# print(sorted(vectorizer_ha.vocabulary_))
vector_ha = vectorizer_ha.transform([df['clean_text'][0]])
print(vector_ha.shape)
print(vector_ha.toarray())

(1, 6)
[[-0.15249857  0.15249857  0.          0.         -0.60999428 -0.76249285]]


In [19]:
vectorizer_tf = TfidfVectorizer()
vectorizer_tf.fit(df['clean_text'])
# print(sorted(vectorizer_tf.vocabulary_))
vector_tf = vectorizer_tf.transform([df['clean_text'][0]])
print(vector_tf.shape)
print(vector_tf.toarray())

(1, 44518)
[[0. 0. 0. ... 0. 0. 0.]]


In [21]:
text = df['clean_text'][0]
docs = df['clean_text']

In [22]:
result = text_to_word_sequence(text)
print(result)

['effici', 'way', 'implement', 'page', 'use', 'linq', 'skip', 'take', 'method', 'page', 'implement', 'page', 'sql', 'queri', 'effici', 'would', 'choos', 'one', 'use', 'sql', 'server', 'asp', 'net', 'mvc', 'linq']


In [23]:
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
# integer encode the document
result = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
print(result)

18
[9, 13, 7, 16, 11, 17, 14, 10, 3, 16, 7, 16, 19, 10, 9, 19, 5, 17, 11, 19, 10, 19, 13, 15, 17]


In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)

In [25]:
tokenizer.word_counts, tokenizer.document_count, tokenizer.word_index, tokenizer.word_docs

(OrderedDict([('effici', 392),
              ('way', 5587),
              ('implement', 2457),
              ('page', 2196),
              ('use', 22829),
              ('linq', 263),
              ('skip', 165),
              ('take', 1236),
              ('method', 4662),
              ('sql', 1399),
              ('queri', 1539),
              ('would', 5758),
              ('choos', 315),
              ('one', 5147),
              ('server', 3435),
              ('asp', 842),
              ('net', 2521),
              ('mvc', 775),
              ('variabl', 1825),
              ('name', 3745),
              ('begin', 448),
              ('mean', 1480),
              ('write', 1860),
              ('first', 2158),
              ('applic', 3635),
              ('see', 2793),
              ('whose', 113),
              ('start', 2294),
              ('underscor', 50),
              ('charact', 837),
              ('specif', 1212),
              ('valid', 1085),
              ('replac'

In [26]:
encoded_docs = tokenizer.texts_to_matrix(docs, mode='count')
encoded_docs

array([[0., 2., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 3., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Word2vec